In [1]:
!pip install -U -qq transformers accelerate bitsandbytes

**Context**

You spend hours perfecting your resume, making sure it outlines your skills and experience in the best possible light. After all, when it comes to job hunting, your resume is your most important tool.

But after all that work, you’re still not getting enough interviews, even for jobs you know you’re qualified for. Why not?

What you might not realize is that your resume usually doesn’t go to a human being after you submit it – it goes to a computer. In fact, there’s a good chance a real person will never see your resume!

That’s because more and more employers are using applicant tracking systems (ATS) to screen resumes. 

What is an ATS? It’s computer software designed to scan resumes for certain keywords and weed out the ones that don’t match the job description.

So if you want your resume to actually make it into the hands of a human being, you need to make sure it’s optimized for the ATS.

In this notebook, we’re going to use NLP and language models to create a resume that can "pass" ATS!

**How applicant tracking systems work?**
There are 4 basic steps to how an applicant tracking system works:

1. A job requisition enters into the ATS. This requisition includes information about the position, such as the job title, desired skills, and required experience.
2. The ATS then uses this information to create a profile for the ideal candidate.
3. As applicants submit their resumes, the ATS parses, sorts, and ranks them based on how well they match the profile.
4. Hiring managers then quickly identify the most qualified candidates and move them forward in the hiring process.

What’s especially important to understand is that recruiters often filter resumes by searching for key skills and job titles.

This means that if you can predict the resume keywords that recruiters will use in their search, you’ll greatly increase your chances of moving on in the hiring process. But you don’t have to guess which keywords to use. All you have to do is analyze the job description to find them.

This notebook automates this process by using AI technology to analyze resume against the job description. It then provides a score that shows how well your resume matches the job description.

**Who uses ATS?**
Over 97% of Fortune 500 companies use ATS while a Kelly OCG survey estimated that 66% of large companies and 35% of small organizations rely on recruitment software. And these numbers continue to grow.

If you’re applying to a large organization, you’ll most likely face an ATS. 

If you’re applying through any online form, you’re applying through an ATS. 

Even job sites like Indeed and LinkedIn have their own built-in ATS.

It’s clear that ATS is here to stay. That’s why it’s so important to use the right keywords and format your resume in a way that makes it easy for ATS software to read.

**How to optimize your resume for an ATS?**
1. Carefully tailor your resume to the job description every single time you apply.
2. Optimize for ATS search and ranking algorithms by matching your resume keywords to the job description.
3. Use both the long-form and acronym version of keywords (e.g. “Master of Business Administration (MBA)” or “Search Engine Optimization (SEO)”) for maximum searchability.
4. Use a chronological or hybrid resume format (avoid the functional resume format).
5. Use a traditional resume font like Helvetica, Garamond, or Georgia.
6. Don’t use headers or footers as the information might get lost or cause a parsing error.
7. Use standard resume section headings like “Work Experience” rather than being cute or clever (“Where I’ve Been”).
8. Use an ATS-friendly resume builder to create your resume.

### Keyword Extraction


### Extracting Job Skills using LLMs

TF-IDF needs a lot of processing and does not work as efficently. Classical NLP tools do not extract all the key words.

The idea of extracting keywords from documents through an LLM is straightforward and allows for easily testing your LLM and its capabilities.

**Why Mistral 7B model?**
Mistral 7B is a 7-billion-parameter language model released by Mistral AI. Mistral 7B is a carefully designed language model that provides both efficiency and high performance to enable real-world applications. Due to its efficiency improvements, the model is suitable for real-time applications where quick responses are essential.

Mistral 7B has demonstrated superior performance across various benchmarks, outperforming even models with larger parameter counts. It excels in areas like mathematics, code generation, and reasoning.

* https://www.linkedin.com/pulse/proof-concept-using-large-language-models-llms-extract-truc-phan-w5vde/
* https://huggingface.co/docs/transformers/main/en/model_doc/llama2
* https://huggingface.co/blog/llama2#how-to-prompt-llama-2
* https://colab.research.google.com/drive/1ge2F1QSK8Q7h0hn3YKuBCOAS0bK8E0wf?usp=sharing
* https://towardsdatascience.com/meta-llama-3-optimized-cpu-inference-with-hugging-face-and-pytorch-9dde2926be5c
* https://www.promptingguide.ai/models/mistral-7b

Experimented with
* 

In [2]:
from huggingface_hub import notebook_login, Repository

# Login to Hugging Face
notebook_login()

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import BitsAndBytesConfig
import torch
import time
import regex
import json

model_name = "mistralai/Mistral-7B-Instruct-v0.2"
#TOKEN = TOKEN

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    llm_int8_enable_fp32_cpu_offload=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(model_name, 
                                             device_map="auto", 
                                             quantization_config=bnb_config)
                                             #token=TOKEN)
tokenizer = AutoTokenizer.from_pretrained(model_name, 
                                          use_fast=True, 
                                          quantization_config=bnb_config)
                                          #token=TOKEN)
# Define a pattern to match JSON object
pattern = regex.compile(r'\{(?:[^{}]|(?R))*\}')

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

#### Extract Skills

In [77]:
skills = """

Expertise in machine learning algorithms, including linear regression, clustering, classification, and recommendation systems
Proven ability to create clear, concise, and compelling visualizations using tools like ggplot2, matplotlib, or plotly, Power BI, Tableau, and Qlik
Conceptual understanding of data analysis fundamentals, encompassing ETL, data warehousing, and unstructured data
A comprehensive grasp of deep learning fundamentals, including activation functions, backpropagation, CNNs, Transformers, transfer learning, and generative models Expertise in evaluating and selecting the most appropriate deep learning model for a given task, including assessing model performance metrics, identifying potential biases, and comparing different model architectures

"""

In [78]:
prompt = f"""
Please respond only in the English language. 
Do not explain what you are doing. 
Do not self reference. 

You are an expert text analyst and researcher. 
Please extract only the most relevant keywords and key phrases from the provided 
{skills}.

Extract keywords for 
data science related skills,  
machine learning techniques, 
data analysis tools,  
programming languages, 
educational qualifications,
experience with number of years,
and soft skills. 

Do not add any unneccesary details. 

Generate a valid JSON object with following key artifacts:
skills: [],
machine learning techniques: [],
tools: [],
programming_languages: [],
education: [],
experience: [],
soft_skills : []

AVOID adding any details if not explicitly mentioned.
Just generate the JSON object without explanation, unique words or duplicates. Be brief.

SKILLS & QUALIFICATIONS
{skills}

"""
start = time.time()
model_inputs = tokenizer(prompt, return_tensors="pt").to("cuda:0")

output = model.generate(**model_inputs,
                          max_new_tokens=1024,
                          repetition_penalty=1.5)

text_string = tokenizer.decode(output[0], 
                       skip_special_tokens=True)

# Find JSON object using regular expression
json_match = pattern.findall(text_string)

print(eval(str(json_match).strip('[]')))
    
end = time.time()
print(f"Time (minutes): {(end - start)}")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


{
"skills": ["expertise", "linear regression","clustering", "classification", "recommendation systems"],
   	     "machinelearningtechniques":["regression", "clusterings", "classifcations", "deep-Learning"],
        "tools": ["ggplot2", "matplotlib", "plotly", "PowerBI", "Tableau", "Qlik"],
         "programmlangages":"NA", // no mention was made about programming langauge(s) used by this person so we leave it as NA (not applicable),
          "education":[],// No educational background information is available here.,
           "experience":[],"soft_skills":["provenability"]}
Time (minutes): 15.322378396987915


#### Extract Job Resposibilities

In [79]:
responsibility = """

Work closely with the client (PO) as well as other Team Leads to clarify the tech requirements and expectations
 Translate complex business problems into actionable data-driven questions or hypotheses. This will involve working with stakeholders to understand the underlying issues and defining the specific questions that need to be answered through data analysis
 Collect data from various sources, identify and address data quality issues, and convert the data into a format suitable for analysis and modeling. This will involve using data wrangling techniques, data cleaning tools, and data quality checks
 Apply statistical methods, data visualization techniques, and machine learning algorithms to uncover hidden patterns, trends, and anomalies within the data. These insights will inform the development of effective models and solutions
Select and apply appropriate machine learning or statistical models to address specific business problems. This involves understanding the problem at hand, choosing the right algorithms, training the models on the prepared data, and evaluating their performance
 Work with engineers to integrate trained models into production environments, ensuring that they can be used to make real-time predictions or decisions. This involves deploying the models, monitoring their performance, and maintaining them over time
Effectively communicate complex data-driven insights and recommendations to stakeholders in a clear, concise, and actionable manner. This involves using storytelling techniques, visualizations, and presentations to effectively convey the findings and their implications for business decisions
Continuously research and stay up-to-date with the latest advancements in data science, including new algorithms, techniques, and tools, and explore emerging technologies and methodologies. This involves attending conferences, reading research papers, and experimenting with new approaches
Suggest and contribute to training and improvement plans regarding analytical data engineering skills, standards, and processes

"""

In [80]:
resp_prompt = f""" 
Please respond only in the English language. 
Do not explain what you are doing. 
Do not self reference. 
You are an expert text analyst. 
Please list all the main responsibilities, tasks and keywords from the following:
{responsibility}

Responsibilities are duties that you will carry out on a regular basis.
Tasks are the specific actions that you will perform.
Extract the keywords mentioned for the position.

DO NOT LIMIT the responsibilities, tasks or keywords.

Generate a valid JSON object with following key artifact:
"responsibilities": [],
"tasks": [],
"keywords": []

Just generate the JSON object without duplicates.
AVOID adding any details if not explicitly mentioned.
Ensure there are no spelling and grammar mistakes.


"""

resp_model_inputs = tokenizer(resp_prompt, return_tensors="pt").to("cuda:0")

resp_output = model.generate(**resp_model_inputs,
                          max_new_tokens=1024,
                          repetition_penalty=1.5)

resp_text_string = tokenizer.decode(resp_output[0], 
                       skip_special_tokens=True)

# Find JSON object using regular expression
resp_json_match = pattern.findall(resp_text_string)

# Print the dictionary
print(eval(str(resp_json_match).strip('[]')))
    
end = time.time()
print(f"Time (minutes): {(end - start)}")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


{
    "responsabilities": [
        "work closely with clients",
        "clarifying technical requirements",
        "translate complex business problems",
        "defining specific questions",
        "collecting data",
        "addressing data quality issues",
        "converting data formats",
        "applying statistical methods",
        "data visualisation techniques",
        "machine learning algorithms",
        "uncover hidden patterns/trends/anomalies",
        "developing effective models/solutions",
        "integrate models into production environment",
        "making real-time predictions/decisions",
        "monitor model performances",
        "maintaining deployed models",
        "communicating results clearly",
        "stay updated with advances in data science",
        "contribute to skill improvements"
    ],
    	"tasks":[],
      "keywords":[
          "working with PO's ",
           "translating business problems",
            "asking data driven questio

**Summary**

In [82]:
profile = """

Analytical and Problem-Solving Skills:
Demonstrated problem-solving and analytical thinking skills, with a proven track record of applying these skills to real-world challenges to identify problems, gather relevant data, and develop creative solutions
Continuous learning mindset, ensuring you stay up-to-date with the latest advancements in deep learning and adapt skills accordingly
Actively participate in the evaluation of new tools for analytical data engineering or data science

"""

In [86]:
sum_prompt = f""" 
Can you provide a comprehensive summary of the given 
{profile}? 

The summary should cover all the key points and main ideas presented in the original text, 
while also condensing the information into a concise and easy-to-understand format. 
Please ensure that the summary includes relevant details and examples that support the main ideas, 
while avoiding any unnecessary information or repetition. 
The length of the summary should be appropriate for the length and complexity of the original text, 
providing a clear and accurate overview without omitting any important information.

Rewrite the summary in first person narrative, active voice, that is professional yet concise.

Generate a valid JSON object with following key artifact:
"summary": ""


"""

sum_model_inputs = tokenizer(sum_prompt, return_tensors="pt").to("cuda:0")

sum_output = model.generate(**sum_model_inputs,
                          max_new_tokens=512,
                          repetition_penalty=1.0)

sum_text_string = tokenizer.decode(sum_output[0], 
                       skip_special_tokens=True)

# Find JSON object using regular expression
sum_json_match = pattern.findall(sum_text_string)

# Print the dictionary
print(eval(str(sum_json_match).strip('[]')))
    
end = time.time()
print(f"Time (minutes): {(end - start)}")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


{
  "summary": "I possess a problem-solving and analytical mindset, demonstrated through my ability to identify challenges, gather data, and develop solutions. I maintain a continuous learning mindset, staying updated with deep learning advancements and adapting skills. I evaluate new tools for analytical data engineering and data science."
}
Time (minutes): 362.9635200500488


**Prompt 1:**

“Could you please provide a concise and comprehensive summary of the given text? The summary should capture the main points and key details of the text while conveying the author's intended meaning accurately. Please ensure that the summary is well-organized and easy to read, with clear headings and subheadings to guide the reader through each section. The length of the summary should be appropriate to capture the main points and key details of the text, without including unnecessary information or becoming overly long.”

**Prompt 2:**
“Could you please provide a summary of the given text, including all key points and supporting details? The summary should be comprehensive and accurately reflect the main message and arguments presented in the original text, while also being concise and easy to understand. To ensure accuracy, please read the text carefully and pay attention to any nuances or complexities in the language. Additionally, the summary should avoid any personal biases or interpretations and remain objective and factual throughout."

### Semantic Similarity 
It is the similarity between two words or two sentences/phrase/text. It measures how close or how different the two pieces of word or text are in terms of their meaning and context.

In [ ]:
from sentence_transformers import SentenceTransformer, util

def semantic_similarity_sbert_base_v2(job,resume, model):
    """calculate similarity with SBERT all-mpnet-base-v2"""
    model = SentenceTransformer(model)
    # Compute embedding for both lists
    embeddings1 = model.encode(job, convert_to_tensor=True)
    embeddings2 = model.encode(resume, convert_to_tensor=True)

    # Compute cosine-similarities
    cosine_scores = util.cos_sim(embeddings1, embeddings2)
     
    return cosine_scores

In [ ]:
resume = """


"""

In [ ]:
semantic_similarity_sbert_base_v2(job,resume, 'all-MiniLM-L12-v1')


### How to Conduct Candidate Analysis

Crafting a comprehensive candidate analysis involves multiple dimensions. Here are the key steps:

1. Resume Keyword Matching: Verify alignment between the resume and job description by looking for overlaps in skills and experience.
2. Competency-Based Evaluation: Scrutinize past achievements using the STAR technique to ensure competencies match those necessary for the role.
3. AIDA Cover Letter Review: Evaluate the cover letter to see if it effectively grabs Attention, maintains Interest, builds Desire, and prompts Action.
4. Fit/Gap Analysis: Determine where a candidate’s skills meet the job prerequisites and where they don't to assess overall compatibility.
5. Growth Potential Assessment: Consider the candidate's past trajectories to estimate their potential for future growth within your startup.

**ChatGPT Prompt for Founders to Create Candidate Analysis**


Using the job description provided, I need a detailed **candidate analysis report** for a job application. This report will assist me in making an informed decision about whether to proceed with the interview process for this candidate.

Here is the job description to use as a benchmark:

[Job Description]

#### Candidate’s Application Analysis:

1. **Resume Keyword Match**: Examine the applicant's resume and extract key skills, experiences, and qualifications. Present these in a bullet-pointed list and note which directly match the job description criteria.

2. **Competency-Based Evaluation**: Analyze the candidate's strongest work achievements. Use the STAR technique to break these down and comment on how these achievements demonstrate competencies required for the job.

3. **Cover Letter AIDA Assessment**: Critique the cover letter using the AIDA model, focusing on how the candidate uses it to illustrate suitability for the role.

4. **Fit-Gap Analysis**: Conduct a fit-gap analysis by creating two lists: one showing where the candidate's skills and experiences match the job requirements ('Fit') and another where they do not align ('Gap').

5. **Growth Potential**: Comment briefly on the candidate's potential for growth and learning within the company based on their career trajectory and achievements presented.

6. **Final Suitability Statement**: Conclude with a suitability statement summarizing whether the candidate should be considered for the role  based on criteria matches, potential growth, and overall fit for the company culture.

Please present your findings in a cohesive markdown format, ensuring each section is clear and well-structured for ease of review.

Candidate's Application:

[Candidate Application]
